In [ ]:
import redis
import time
import pickle
import json

In [ ]:
def getLUT() :
    """ 
        LUT de test avec 3 listes et 5 règles: 
            L1 = {0, 1, 3, 4}, 
            L2 = {0, 2, 3}, 
            L3 = {1, 2, 4}
    """
    return {
            pickle.dumps([False, False, False, False, False]) : [],
            pickle.dumps([False, True, True, False, True]) : ["L3"],
            pickle.dumps([True, False, True, True, False]) : ["L2"],
            pickle.dumps([True, False, True, True, True]) : ["L2", "L3"],
            pickle.dumps([True, True, False, True, True]) : ["L1"],
            pickle.dumps([True, True, True, True, True]) : ["L1", "L2", "L3"]
    }

In [ ]:
def computeAttack(tests) :
    lut = getLUT()
    return lut[pickle.dumps(tests)]

In [ ]:
# Connect to Redis

pool = redis.ConnectionPool(host="queue", port=6379, db=0)
r = redis.Redis(connection_pool=pool)
p = r.pubsub()

p.subscribe("Queue")

p.get_message()
time.sleep(0.5)

def getData(p) :
    """ 
    Get incoming items in the queue as a generator
    """
    for mes in p.listen() :
        print(mes)
        if type(mes["data"]) == type(b'') :
            yield json.loads(mes["data"])


In [ ]:
for tests in getData(p) :
    # Compute the test results and send them to be logged
    r.publish(  channel="Log", 
                message=pickle.dumps(
                    (   
                        tests["id"], 
                        computeAttack(tests["results"])
                    )
                )
            )